<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDS0701ESCoursera21685115-2022-01-01"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmentar y Agrupar Vecindarios en Nueva York</font></h1>


## Introducción

En este laboratorio aprenderá a convertir direcciones en sus valores equivalentes de latitud y longitud. Además, utilizará la API de FourSquare para explorar los vecindarios de la ciudad de Nueva York. Usará la función **explore** para obtener las categorias de lugares mas comunes en cada vecindario, después utilizará esta característica para agrupar los vecindarios en agrupaciones. Usará el algoritmo de agrupamiento *k*-means para completar esta tarea. Finalmente usará la librería Folium para visualizar los vecindarios de la ciudad y sus agrupaciones emergentes.


## Indice

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="https://#item1">Descargar y Explorar el Conjunto de Datos</a>

2.  <a href="https://#item2">Explorar Vecindarios en la Ciudad de Nueva York</a>

3.  <a href="https://#item3">Analizar cada Vecindario</a>

4.  <a href="https://#item4">Agrupaciones de Vecindarios</a>

5.  <a href="https://#item5">Examinar Agrupaciones</a>\ </font>
    </div>


Antes de obtener los datos y empezar a explorarlos, vamos a descargar todas las dependencias necesarias.


In [ ]:
import numpy as np # librería para manejar datos vectorizados

import pandas as pd # librería para análisis de datos
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # librería para manejar archivos JSON 

#!conda install -c conda-forge geopy --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare 
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas

# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

print('Libraries imported.')

<a id='item1'></a>


## 1. Descarga y Exploración del Conjunto de Datos


El vecindario tiene un total de 5 distritos y 306 barrios. Para poder segmentar los barrios y explorarlos necesitaremos un conjunto de datos que contenga los 5 distritos y los barrios que existen en cada uno además de la latitud y longitud de cada barrio.

Por suerte, este conjunto existe gratuitamente en la web. Pruebe a encontrar con libertad este conjunto por su cuenta, aquí esta el enlace: [https://geo.nyu.edu/catalog/nyu\_2451\_34572](https://geo.nyu.edu/catalog/nyu\_2451\_34572?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDS0701ESCoursera21685115-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)


Para su conveniencia he descargado los archivos y los he puesto en el servidor para que unicamente ejecute el comando `wget` y acceda a ellos. Hagamoslo.


In [ ]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

#### Cargar y explorar lo datos


Vamos a cargar lo datos.


In [ ]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Veamos rapidamente los datos.


In [ ]:
newyork_data

Observe como todos los datos relevantes estan en la llave *features*. la cual es una lista de los barrios. Definamos una nueva variable que incluya estos datos.


In [ ]:
neighborhoods_data = newyork_data['features']

Veamos el primer elemento de la lista


In [ ]:
neighborhoods_data[0]

#### Transformar los datos en un dataframe *pandas*


La próxima tarea es esencialmente transformar estos datos de diccionarios de Python anidados en un dataframe. Empecemos por crear un dataframe vacio.


In [ ]:
# definir las columnas del datagrama
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# inicializar el dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Observe el dataframe vacio para confirmar las columnas como estaba previsto.


In [ ]:
neighborhoods

Hagamos una iteración a traves de los datos y llenemos el dataframe una fila a la vez.


In [ ]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Examinemos rapidamente el dataframe resultante.


In [ ]:
neighborhoods.head()

Y aseguremonos de que el conjunto de datos tiene los 5 distritos y los 306 barrios.


In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

#### Utilice la librería geopy para obtener la latitud y la longitud de la Ciudad de Nueva York


Para poder definir una instancia del geocoder necesitaremos definir un user_agent. Nombraremos a nuestro agente <em>ny_explorer</em>, como se muestra a continuación.


In [ ]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

#### Genere un mapa de Nueva York con los barrios super puestos por encima.


In [ ]:
# crear un mapa de Nueva York utilizando los valores de latitud y longitud
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** es una gran librería de visualización. Pruebela para hacer zoom en el mapa de arriba y haga clic en cada marcador circular para mostrar el nombre del barrioy su respectivo distrito.


Sin embargo, para efectos de ilustración, simplifiquemos el mapa de arriba y segmentemos y agrupemos unicamente los barrios en Manhattan. Dividamos el dataframe original y generemos uno nuevo con los datos de Manhattan.


In [ ]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

Obtengamos las coordenadas geográficas de Manhattan.


In [ ]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

Como se hizo con Nueva York, visualicemos los barrios de Manhattan.


In [ ]:
# crear un mapa de Manhattan usando los valores de latitud y longitud
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# añadir los marcadores al mapa
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

A continuación vamos a empezar a utilizar el API de FourSquare para explorar los barrios y segmentarlos.


#### Definir la versión y credenciales de FourSquare


In [ ]:
CLIENT_ID = 'your-client-ID' # su ID de Foursquare
CLIENT_SECRET = 'your-client-secret' # Secreto de Foursquare
VERSION = '20180605' # versión de la API de Foursquare
LIMIT = 100 # Un valor límite para la API de Foursquare

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Exploremos el primer barrio del dataframe.


Obtener el nombre el barrio.


In [ ]:
manhattan_data.loc[0, 'Neighborhood']

Obtener la latitud y longitud del barrio.


In [ ]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # latitud del barrio 
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # longitud del barrio

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # nombre del barrio

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

#### Obtengamos los 100 sitios en Marble Hill dentro de un radio de 500 metros.


Primero, generemos una URL de solicitud GET. Nombre su URL **url**.


In [ ]:
# escriba aquí su respuesta



Haga doble clic **aquí** para ver la solución.

<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 


Enviar la solicitud GET y examinar los resultados.


In [ ]:
results = requests.get(url).json()
results

Del laboratorio anterior de FourSquare sabemos que toda la información esta en la llave *items*. Antes de avanzar, usemos la función **get_category_type** del laboratorio anterios.


In [ ]:
# función para extraer la categoria del sitio
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Ahora estamos listos para limpiar el objeto json y estructurarlo en un dataframe *pandas*.


In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # objeto JSON

# filtrar columnas
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filtrar la categoría para cada fila
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# limpiar columnas
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

¿Cuantos sitios nos regresó FourSquare?


In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

<a id='item2'></a>


## 2. Explorar Barrios en Manhattan


#### Vamos a crear una función que repita el mismo proceso para todos los barrios de Manhattan


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # solicitud GET
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # regresa solo información relevante de cada sitio cercano
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.


In [ ]:
# escriba aquí su respuesta


Haga doble clic **aquí** para ver la solución.

<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 


#### Revisemos el tamaño del dataframe resultante


In [ ]:
print(manhattan_venues.shape)
manhattan_venues.head()

Revisemos cuantos sitios se regresarón para cada barrio


In [ ]:
manhattan_venues.groupby('Neighborhood').count()

#### Encontremos cuantas categorías únicas se pueden conservar de todos los sitios regresados


In [ ]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

<a id='item3'></a>


## 3. Analizar Cada Barrio


In [ ]:
# codificación
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# mover la columna de barrio a la primer columna
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

Examinemos el tamaño del nuevo dataframe


In [ ]:
manhattan_onehot.shape

#### Agrupemos las filas por barrios tomando la média de la frecuancia de la ocurrencia de cada categoría


In [ ]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

#### Confirmemos el nuevo tamaño


In [ ]:
manhattan_grouped.shape

#### Imprimamos cada barrio junto con los 5 sitios mas comunes


In [ ]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Pongamos eso en el dataframe


Primero escribamos una función para ordenar los sitios en orden descendente.


In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Generemos el nuevo dataframe y mostremos los primeros 10 sitios de cada barrio.


In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<a id='item4'></a>


## 4. Barrios Agrupados


Ejecutemos *k*-means para agrupar los barrios en 5 agrupaciones.


In [ ]:
# establecer el número de agrupaciones
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:10] 

Generemos un nuevo dataframe que incluya la agrupación asi como los 10 sitios mas populares de cada barrio.


In [ ]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# juntar manhattan_grouped con manhattan_data 
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # revisar las ultimas columnas

Finalmente visualicemos las agrupaciones resultantes


In [ ]:
# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examinar Agrupaciones


Ahora puede examinar cada agrupación y determinar las categorias del sitio que distingue a cada agrupación. En base a las categorias definidas usted puede asignar un nombre a cada agrupación. Dejaré este ejercicio para usted.


#### Agrupación 1


In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Agrupación 2


In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Agrupación 3


In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Agrupación 4


In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Agrupación 5


In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

### Gracias por terminar este laboratorio

Este cuaderno fue creado por [Alex Aklson](https://www.linkedin.com/in/aklson?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDS0701ESCoursera21685115-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) y [Polong Lin](https://www.linkedin.com/in/polonglin?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDS0701ESCoursera21685115-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). Espero que haya encontrado este laboratorio de su interes y educativo. Tómese la libertad de contactar conmigo para cualquier duda o aclaración.


Este cuaderno forma parte del curso en **Coursera** llamado *Applied Data Science Capstone*. Si accede a este cuaderno desde afuera del curso, puede tomarlo en línea haciendo clic [aquí](http://cocl.us/DP0701EN_Coursera_Week3\_LAB2).


<hr>

Copyright © 2018 [Cognitive Class](https://cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDS0701ESCoursera21685115-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
